In [1]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver_manager

In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import io
import gzip
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import glob
import multiprocessing
from multiprocessing import Pool

driver = webdriver.Chrome(ChromeDriverManager().install())

url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
driver.get(url)

segment = "Equity"
status = "Active"

segment_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddSegment"]')
segment_element.send_keys(segment)

status_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlStatus"]')
status_element.send_keys(status)

submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
submit.send_keys(Keys.RETURN)


download = driver.find_element_by_xpath(
    '/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i')
download.click()
time.sleep(5)
driver.quit()

df_equity = pd.read_csv("C:/Users/MEESALA ARAVIND/Downloads/Equity.csv")
security_numbers = df_equity["Security Code"].tolist()
security_n = df_equity["Security Name"].tolist()
security_names = dict(zip(security_numbers, security_n))

store_cache = {}

def get_stocks(nums):
    
    def get_date(d,m,y):
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]')   
        for opt in year.find_elements_by_tag_name("option"):
            if opt.text == y:
                opt.click()
                break
        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]')   
        for opt in month.find_elements_by_tag_name("option"):
            if opt.text == m:
                opt.click()
                break
        days = driver.find_element_by_xpath('/html/body/div[1]/table')
        for row in days.find_elements_by_tag_name('tr'):
            for col in row.find_elements_by_tag_name('td'):
                if col.text == d:
                    row.click()
                    return

    def traverse_stock(security):
        if security in store_cache:
            return None
#         print(security, type(security))
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)
        time.sleep(2)
        try:
            response = driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[3]/div/div/table/tbody/tr[2]/td/div/div[2]/div/div/div[2]/div[1]/table/tbody/tr/td/div/div/div/ul/li")
        except Exception as e:
            pass

        if response.text == "No Match Found":
            return None
        else:
            from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
            from_date.clear()
            from_date.click()
            get_date("31","Dec","2015")
            to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
            to_date.clear()
            to_date.click()
            get_date("31","Dec","2020")
            submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
            submit.click()
            time.sleep(2)
            store_stock(security)
            store_cache[security] = True


    def store_stock(security):
        download_stock = driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[4]/div/div/div[1]/div[4]/div[2]/div/span/a/i")
        download_stock.click()
    
#     print(nums)
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
    url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
    driver.get(url)
    traverse_stock(str(nums))
    time.sleep(8)
    driver.quit()




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\laksh\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


In [ ]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import io
import gzip
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import glob
import multiprocessing
from multiprocessing import Pool
#from ipynb.fs.full.BSE_Scraping_defs_Copy1 import *


if __name__ == "__main__":
    p = Pool(8)
    p.map(get_stocks, security_numbers)